In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pickle
import random
import psycopg2

In [2]:
with open('output/SPARSE_MATRIX.pkl', 'rb') as f:
        smat = pickle.load(f)

In [3]:
products = pd.read_csv('instacart_2017_05_01/products.csv').set_index('product_id')[['product_name']]
products.head()

,product_name
product_id,
1,Chocolate Sandwich Cookies
2,All-Seasons Salt
3,Robust Golden Unsweetened Oolong Tea
4,Smart Ones Classic Favorites Mini Rigatoni Wit...
5,Green Chile Anytime Sauce


In [4]:
sparse_frame = pd.DataFrame.sparse.from_spmatrix(smat)

In [5]:
def search(df, item):
    return df[df.product_name.str.contains(item)]

In [6]:
search(products, 'meat')

,product_name
product_id,
665,Imitation Crabmeat
2595,"Tender white meat chicken, vegetables & rice i..."
5124,Freshly made pasta layered with seasoned meat ...
5568,Lunchmeat
6212,"White meat chicken, shredded cabbage, julienne..."
8344,A golden crust surrounding pieces of all white...
11919,Tender white meat chicken in a red chile lime ...
15125,Tender white meat turkey & stuffing in a homes...
16600,"With seasoned meat sauce, tomato sauce & real ..."


In [7]:
def get_product(df, pid):
    return df.loc[pid].product_name

def get_all_cocart(sp_mat, pid, top=10):
    # sp_mat[pid] column product count, row product count: sp_mat.loc[pid] (index is the product id) 
    # We are doing this because we have triangular matrix
    return pd.concat((sp_mat[pid], sp_mat.loc[pid])).dropna().nlargest(top)


def get_cocart(df, pid):
    json = {}
    for idx, val in get_all_cocart(sparse_frame, pid).iteritems():
        if val > 0:
            json[idx] =  val
    json = {get_product(df, pid): json}
    return json

In [8]:
get_cocart(products, 30)

{'Three Cheese Ziti, Marinara with Meatballs': {28076: 16,
  49275: 15,
  44792: 13,
  24852: 8,
  28204: 8,
  40454: 7,
  46674: 7,
  4: 7,
  36: 6,
  4605: 6}}

In [9]:
def get_all(df, idxx):
    all_responses = [get_cocart(df, idx)[get_product(df, idx)] for idx in idxx]
    d = {}
    for resp in all_responses:
        d = {**d, **resp}
    return d

In [10]:
search(products, 'Oreo')

,product_name
product_id,
591,Oreo Cookies and Cream Chocolate Frozen Dairy ...
796,Chocolate Reese's Peanut Butter Cup Creme Oreo
2409,Oreo Mini 20 Count Multipack 20oz
3126,Oreo Cookie n' Cream Yogurt
3260,Chips Ahoy!/Nutter Butter/Oreo Cookies
4057,Oreo Instant Cookies 'n Cream Pudding & Pie Fi...
6681,11.3 Oz. Oreo Fudge Creme Double Chocolate
9278,Oreo Brownie Batter Cookies
9389,Oreo Mini/Nutter Butter Bites/Mini Chips Ahoy!...


In [11]:
f = search(products, 'Oreo').index[:3]
FF= get_all(products, f)

In [12]:
for k in FF:
    print(get_product(products, k))

Banana
Unsweetened Iced Coffee
Honeycrisp Apples
Original Pepperoni Pizza
2% Reduced Fat Milk
Fat Free Skim Milk
Original Strawberry Banana Yogurt
Original Strawberry Mango Yogurt
Baby Spinach
Dark Chocolate Almond Milk
Strawberries
Original Nooks & Crannies English Muffins
Double Stuf Oreo Sandwich Cookies
Classic Soda
Whole Milk
Mini Peeled Carrots
Green Onions
Hass Avocados
Oreo Chocolate Sandwich Cookies
Classic Mix Cookies & Crackers
Oreo Mini/Nutter Butter Bites/Mini Chips Ahoy! Cookies
of Hanover 100 Calorie Pretzels Mini
Fridge Pack Cola
Original Superfood Fruit Smoothie Blend
Mo' Beta Fruit Smoothie Blend
Pepperidge Farm® Goldfish Xtra Cheddar Baked Snack Crackers
Chocolate Chip Cookies
Cereal with Real Strawberries


In [13]:
with open('output/sparse_frame.pkl', 'wb') as f:
    pickle.dump(sparse_frame, f)